# RAG Thai
ใช้ ChatGPT ช่วยสร้างโค้ดบางส่วนยังไม่แน่ใจว่ามันทำงานถูกหรือเปล่า
- Thai: ลองเพิ่มการจัดการภาษาไทย normalize tokenize split chunk
- embeding : ใช้แบบ Opensource เช่น intfloat/multilingual-e5-base
- Vector : FAISS สำหรับ Symantic Search
- Full text : [rank_bm25](https://pypi.org/project/rank-bm25/) สำหรับ Full Text Search ตัวนี้ไม่รองรับการจัดการ text เราต้องทำเอง(ตัดคำ ตัดประโยค normalize)
- Generate : ใช้ Gemini 

ให้ผลแม่นยำกว่า [simple_rag.ipynb](./simple_rag.ipynb) ทดสอบจากการค้นร้านอาหาร

In [75]:
!uv add numpy google-genai pythainlp sentence-transformers faiss-cpu rank-bm25 ipywidgets

Resolved 162 packages in 7ms
Audited 158 packages in 0.07ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [76]:
import os
import json
from google import genai
from pythainlp import word_tokenize
from pythainlp.util import normalize
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import torch

if not os.environ.get("GEMINI_API_KEY"):
    raise ValueError("Please set the GEMINI_API_KEY environment variable")

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

client = genai.Client()


## Text utils
ฟังก์ชั่นสำหรับจัดการข้อความ normalize, token, split, chunk

In [77]:
def th_normalize(text: str) -> str:
    # ลบซ้ำวรรณยุกต์/สระยาว, normalize รูปแบบ, ตัดช่องว่างเกิน
    t = normalize(text or "")
    t = re.sub(r"[^\S\r\n]+", " ", t).strip()
    # ลดเสียงลากยาว "มากกกก" -> "มาก"
    t = re.sub(r"(.)\1{2,}", r"\1", t)
    return t

def th_tokenize(text: str):
    # เลือก engine 'newmm' ซึ่งแม่นยำดีสำหรับไทย
    return word_tokenize(text, engine="newmm")

def sent_split_th(text: str):
    # แยกเป็นประโยคแบบง่าย ๆ ตามเครื่องหมายวรรคตอนไทย/อังกฤษ
    # (สำหรับงานจริง อาจใช้ประโยคจาก Markdown heading/ย่อหน้าร่วมด้วย)
    return re.split(r"(?<=[\.\?\!]|[ๆฯ])\s+", text.strip())

# -------- Chunking ตามประโยค + ขนาดโทเคนคร่าว ๆ --------
def chunk_by_sentences(text, max_chars=600, overlap_chars=80):
    sents = [s for s in sent_split_th(th_normalize(text)) if s]
    chunks = []
    buf = ""
    for s in sents:
        if len(buf) + len(s) + 1 <= max_chars:
            buf = (buf + " " + s).strip()
        else:
            if buf:
                chunks.append(buf)
            # overlap
            if overlap_chars > 0 and chunks:
                tail = chunks[-1][-overlap_chars:]
                buf = (tail + " " + s).strip()
            else:
                buf = s
    if buf:
        chunks.append(buf)
    return chunks


ทดสอบ text utils

In [78]:
# test_thai_word = "กรุงเทพฯ 5 มี.ค. - ธนาคารแห่งประเทศไทย (ธปท.) \nเตือนประชาชนระวังมิจฉาชีพแอบอ้างเป็นเจ้าหน้าที่ธนาคารหรือหน่วยงานรัฐ ส่ง SMS หรือ LINE ที่มีลิงก์ปลอม เพื่อหลอกให้โอนเงินหรือติดตั้งแอปพลิเคชันอันตราย พร้อมแนะวิธีป้องกันตัวเองจากมิจฉาชีพ"
test_thai_word = "สวัสดีมากๆ!เลย    จ้าาาา"
print("normalize:",th_normalize(test_thai_word) )
print("tokenize:",th_tokenize(test_thai_word))
print("split:",sent_split_th(test_thai_word))
print("norm+token:",th_tokenize(th_normalize(test_thai_word)))
print("chunk_by_sentences:",chunk_by_sentences(test_thai_word))

normalize: สวัสดีมากๆ!เลย จ้า
tokenize: ['สวัสดี', 'มาก', 'ๆ', '!', 'เลย', '    ', 'จ้า', 'าาา']
split: ['สวัสดีมากๆ!เลย    จ้าาาา']
norm+token: ['สวัสดี', 'มาก', 'ๆ', '!', 'เลย', ' ', 'จ้า']
chunk_by_sentences: ['สวัสดีมากๆ!เลย จ้า']


## Embedding
ตัวอย่างนี้ใช้ intfloat/multilingual-e5-base หรือ BAAI/beg-m3 รองรับหลายภาษารวมทั้งภาษาไทย ตั้งไว้แบบ CPU เพื่อจะได้ไม่มีปัญหากับเครื่องที่ไม่มีการ์ดจอหรือไม่ได้ตั้งค่าไว้

In [79]:
#embed_model = SentenceTransformer("intfloat/multilingual-e5-base",device='cpu')
embed_model = SentenceTransformer("BAAI/bge-m3",device=device)
def embed(texts):
    return embed_model.encode([f"{t}" for t in texts], normalize_embeddings=True,device=device)
    

In [80]:
def create_chunk_records(docs):
    # สร้างชิ้นข้อมูล (chunks)
    chunk_records = []
    for d in docs:
        chunks = chunk_by_sentences(d["description"], max_chars=500, overlap_chars=50)
        for i, c in enumerate(chunks):
            chunk_records.append({
                "doc_id": d["id"],
                "title": d["name"],
                "chunk_id": f'{d["id"]}-{i}',
                "text": c,
            })
    return chunk_records
def create_embedding(chunk_records):
    emb_matrix = embed([f"{r["title"]}:\n{r["text"]}"  for r in chunk_records]).astype("float32")
    # ดัชนีเวคเตอร์ FAISS
    d = emb_matrix.shape[1]
    index = faiss.IndexFlatIP(d)  # ใช้ cosine-sim กับเวคเตอร์ normalized = dot product
    index.add(emb_matrix)
    return index

## Indexing Data
โหลดข้อมูลจากไฟล์แล้วสร้าง  bm25, vector และ รายการข้อมูล

In [81]:
def reload_data(file_path):
    docs=[]
    with open(file_path) as f:
        docs = json.load(f)
    chunk_records = create_chunk_records(docs)
    embedding_index = create_embedding(chunk_records)
    bm25_corpus_tokens = [th_tokenize(r["text"]) for r in chunk_records]
    bm25_index = BM25Okapi(bm25_corpus_tokens)
    return embedding_index, bm25_index, chunk_records


ทดสอบการโหลดข้อมูล แล้วทำการค้นหาแบบ Full Text(BM25) และ Similar(Vector) พร้อมแสดงคะแนน 
- data1.json (ภาษาอังกฤษ) ค้นคำว่า "Thai" ผลค่อนข้างใกล้เคียงกัน แต่ถ้าใช้ "ไทย" BM25 จะได้ score 0 ส่วน vector ถูกบางตัว
- data2.json (ภาษาไทย) ค้นคำว่า "มิจฉาชีพ" ผลค่อนข้างใกล้เคียงกัน ถ้าใช้ "Phishing" BM25 จะได้ score 0 ส่วน Vector ให้ผลดีกว่า

In [86]:
## Test search from index
embeded_index,b25_index,chunks_docs = reload_data("./data2.json")# reload index
# print(json.dumps(chunks_docs, indent=2)) 
q_norm = th_normalize("มิจฉาชีพ") # normalize Query

# Full Text Search with socre
q_tokens = th_tokenize(q_norm) # normalize and tokenize query
bm25_scores = b25_index.get_scores(q_tokens)
bm25_top_idx = np.argsort(bm25_scores)[::-1][:3]
# print([i['title'] for i in b25_index.get_top_n(q_tokens, chunks_docs, n=3)])
print("BM25:",[f"{bm25_scores[i]}:{chunks_docs[i]['title']}" for i in bm25_top_idx])

# Similar search with score
q_emb = embed_model.encode([f"query: {q_norm}"], normalize_embeddings=True).astype("float32")
sim, idx = embeded_index.search(q_emb, 3)  # cosine-sim (dot)
print("Vector:",[f"{i2}:{chunks_docs[i1]['title']}" for i1,i2 in zip(idx[0],sim[0])])

BM25: ['1.5881503124135494:การใช้งาน ATM', '1.2412011622247787:คำเตือนภัยหลอกลวง', '0.0:ทำอย่างไรเพื่อหลีกเลี่ยง Spyware']
Vector: ['0.49123451113700867:คำเตือนภัยหลอกลวง', '0.46242836117744446:การใช้งาน ATM', '0.44394198060035706:ระวัง SMS/ลิงก์ปลอม']


## Hybrid Search
รวมการค้นหาสองแบบเข้าด้วยกัน ถ่วงน้ำหนักด้วย alpha การค้นหาแบบ Similar (Vector) อาจจะไม่ค่อยแม่นยำนักกับภาษาไทยที่ค่อนข้างซับซ้อน จะใช้การค้นหาแบบ Full Text(BM25) ร่วมด้วย

In [89]:
def hybrid_search(query,vector_index,bm25_index,chunk_records, top_k=6, alpha=0.5):
    """
    ค้นหาแบบ BM25 และ vector ถ่วงน้ำหนักด้วย alpha: ถ้า 1.0 เน้น vector ล้วน, ถ้า 0.0 เน้น BM25 ล้วน
    """
    # Vector
    q_norm = th_normalize(query)
    q_emb = embed_model.encode([f"query: {q_norm}"], normalize_embeddings=True).astype("float32")
    sim, idx = vector_index.search(q_emb, top_k*3)  # cosine-sim (dot)
    vec_scores = np.zeros(len(chunk_records))
    vec_scores[idx[0]] = sim[0]
    #BM25
    q_tokens = th_tokenize(q_norm)
    bm25_scores = bm25_index.get_scores(q_tokens)
    bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k*3]  # ขยายเผื่อรวมกับเวคเตอร์

    # รวมคะแนน (normalize แบบ min-max)
    def norm(x):
        x = np.array(x, dtype=np.float32)
        if x.max() - x.min() < 1e-8:
            return np.zeros_like(x)
        return (x - x.min()) / (x.max() - x.min())

    bm25_mask = np.zeros(len(chunk_records)); bm25_mask[bm25_top_idx] = 1
    union_idx = np.where((bm25_mask + (vec_scores>0))>0)[0]

    score_b = norm(bm25_scores[union_idx])
    score_v = norm(vec_scores[union_idx])
    score = alpha*score_v + (1-alpha)*score_b

    order = union_idx[np.argsort(score)[::-1][:top_k]]
    results = []
    for i in order:
        r = chunk_records[i]
        results.append({
            "chunk_id": r["chunk_id"],
            "doc_id": r["doc_id"],
            "title": r["title"],
            "text": r["text"],
            "score": float(score[np.where(union_idx==i)][0]),
        })
    return results

## Generate
เมื่อนำผลจาก hybrid_search ที่อาจจะผิดพลาดบ้าง มาสร้าง prompt เพื่อประมวลต่อด้วย LLM จะได้คำตอบที่ตรงกับที่ต้องการมากขึ้น

In [90]:
# สร้างคำตอบโดยยึดตามหลัก "ใช้เฉพาะจากบริบท" ใช้ contexts เหมือนจะดีกว่า context_text
def generate_response(query, contexts,max_ctx_chars=1200):
    """Generate a response using Gemini based on the query and retrieved context"""
    context_text = ""
    for c in contexts:
        if len(context_text) + len(c["text"]) + 2 <= max_ctx_chars:
            context_text += f"- {c['text']}\n"

    prompt = f"""
You are a helpful assistant .
If the query asks for something not in the provided information, politely indicate 
that you don't have that specific information but suggest the closest alternatives.
Please provide a helpful response that directly answers the user's query based on information below.

USER QUERY: {query}

INFORMATION:
{contexts}

"""
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt,
    )
    answer = response.text
    return prompt, answer


Reload Data

In [91]:
embeded_index,b25_index,chunks_docs = reload_data("./data1.json")# reload index

data2.json : doc9 ถูก Chunking ออกมาได้ไม่ดีซักเท่าไหร่

In [94]:
chunks_docs

[{'doc_id': '001',
  'title': 'La Dotta',
  'chunk_id': '001-0',
  'text': 'Authentic Italian restaurant specializing in handmade pasta dishes using imported ingredients from Italy. Located in Thonglor district of Bangkok.'},
 {'doc_id': '002',
  'title': 'Peppina',
  'chunk_id': '002-0',
  'text': 'Neapolitan pizza restaurant with wood-fired ovens imported from Italy. Features traditional Italian dishes and a wide selection of Italian wines.'},
 {'doc_id': '003',
  'title': "L'Oliva",
  'chunk_id': '003-0',
  'text': 'High-end Italian dining with focus on Northern Italian cuisine. Offers homemade pasta, risotto, and seafood specialties in the heart of Sukhumvit.'},
 {'doc_id': '004',
  'title': 'Appia',
  'chunk_id': '004-0',
  'text': 'Roman-inspired trattoria serving hearty Italian comfort food including porchetta and homemade pasta in a rustic setting in Sukhumvit Soi 31.'},
 {'doc_id': '005',
  'title': 'Pizza Massilia',
  'chunk_id': '005-0',
  'text': 'Upscale Italian restaurant

RAGS ลองปรับ user_query และ alpha ว่าผลแตกต่างกันหรือไม่

In [ ]:
# user_query = "ถ้าโอนเงินผิดบัญชีควรทำอย่างไร และควรระวังมิจฉาชีพแบบไหน?"
user_query = "ร้านไหนบ้างขายอาหารไทยหรืออิตาลี"
# user_query = "ถ้าได้ SMS น่าสงสัยควรทำอย่างไร"
results = hybrid_search(user_query,embeded_index,b25_index,chunks_docs, top_k=6, alpha=1.0)
# chunks_docs มีข้อมูลไม่ครบทั้งหมดเช่นที่อยู่ของร้านอาหาร น่าจะเพิ่มเข้าไปด้วยตอนทำ RAG จะได้มีข้อมูลส่วนนี้ด้วย
prompt, answer = generate_response(user_query, results)
print("Hybrid Search Top Hits: ")
for i, r in enumerate(results):
    print(f"{i+1} {r['title']} ({r['score']:.3f})")
    print(f"{r['text'][:120]}...")

Hybrid Search Top Hits: 
1 Isaan Der (1.000)
Northeastern Thai cuisine featuring grilled meats, sticky rice, and spicy salads served in a casual atmosphere near Asok...
2 Pad Thai Ekkamai (1.000)
Local favorite serving authentic pad thai and other traditional Thai noodle dishes in the trendy Ekkamai neighborhood....
3 Appia (1.000)
Roman-inspired trattoria serving hearty Italian comfort food including porchetta and homemade pasta in a rustic setting ...
4 Som Tam Nua (1.000)
Popular Thai restaurant specializing in Northeastern Thai cuisine, especially spicy papaya salad and grilled chicken. Lo...
5 Gianni's (1.000)
Classic Italian restaurant in Sukhumvit offering traditional dishes from various regions of Italy, with an extensive win...
6 Pizza Massilia (1.000)
Upscale Italian restaurant specializing in gourmet pizzas with premium imported ingredients and authentic Italian recipe...


In [126]:
print("Prompt:\n", prompt)

Prompt:
 
You are a helpful assistant .
If the query asks for something not in the provided information, politely indicate 
that you don't have that specific information but suggest the closest alternatives.
Please provide a helpful response that directly answers the user's query based on information below.

USER QUERY: ร้านไหนบ้างขายอาหารไทยหรืออิตาลี

INFORMATION:
[{'chunk_id': '009-0', 'doc_id': '009', 'title': 'Isaan Der', 'text': 'Northeastern Thai cuisine featuring grilled meats, sticky rice, and spicy salads served in a casual atmosphere near Asok.', 'score': 1.0}, {'chunk_id': '008-0', 'doc_id': '008', 'title': 'Pad Thai Ekkamai', 'text': 'Local favorite serving authentic pad thai and other traditional Thai noodle dishes in the trendy Ekkamai neighborhood.', 'score': 1.0}, {'chunk_id': '004-0', 'doc_id': '004', 'title': 'Appia', 'text': 'Roman-inspired trattoria serving hearty Italian comfort food including porchetta and homemade pasta in a rustic setting in Sukhumvit Soi 31.',

In [128]:
print("ถาม:",user_query,"\n ตอบ:\n",answer)

ถาม: ร้านไหนบ้างขายอาหารไทยหรืออิตาลี 
 ตอบ:
 ร้านอาหารที่ขายอาหารไทยหรืออิตาลีตามข้อมูลที่มี ได้แก่:

**อาหารไทย:**
*   **Isaan Der** (อาหารไทยอีสาน)
*   **Pad Thai Ekkamai** (ผัดไทยและอาหารเส้นไทยดั้งเดิม)
*   **Som Tam Nua** (อาหารไทยอีสาน เน้นส้มตำและไก่ย่าง)

**อาหารอิตาลี:**
*   **Appia** (อาหารอิตาเลียนสไตล์โรมัน)
*   **Gianni's** (อาหารอิตาเลียนคลาสสิก)
*   **Pizza Massilia** (พิซซ่าอิตาเลียนรสเลิศ)


## สรุป
- BM25 จะเหมาะกับข้อมูลที่มีคำในภาษานั้นๆตรงๆ ซึ่งการค้นหาส่วนใหญ่มักเป็นแบบนั้น
- Vector เข้าใจคำที่มีความหมายคล้ายกันหรือคนละภาษาได้ เหมาะเวลาที่เราไม่รู้คำที่ชัดเจน
- Hybrid Search นำข้อดีของ BM25 และ Vector มาใช้ร่วมกันทำให้มีโอกาสค้นหารูปแบบต่างๆได้
- RAGS Hybrid เพิ่มความแม่นยำในการค้นหามากขึ้น